In [1]:
import json
import pandas as pd
import numpy as np
import statistics

# loading XGBoost
with open("output/output XGBoost without hyperparameters.json", 'r') as f:
  data = json.load(f)

# sort dataset by normalization
df_XGBoost = pd.json_normalize(data, record_path =['measurements'])

# save minimum percentage to be dropped in extra variable
df_XGBoost["drop_columns"] = df_XGBoost["best_params.drop_columns__minimum_percentage_to_be_dropped"]
 

In [35]:
# calculate main values of interest
df_XGBoost["auc_std"] = df_XGBoost.apply(lambda row: np.std(row["auc"]), axis =1)
df_XGBoost["pre_mean"] = df_XGBoost.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_XGBoost["rec_mean"] = df_XGBoost.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_XGBoost["f1_std"] = df_XGBoost.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_XGBoost["f1_mean"] = df_XGBoost.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_XGBoost["type2"] = df_XGBoost.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
# Confidence Intervalls
df_XGBoost["auc_confl"] = df_XGBoost["auc_mean"]- 1.96 * (df_XGBoost["auc_std"] / np.sqrt(10))
df_XGBoost["auc_confu"] = df_XGBoost["auc_mean"]+ 1.96 * (df_XGBoost["auc_std"] / np.sqrt(10))
df_XGBoost["f1_confl"] = df_XGBoost["f1_mean"]- 1.96 * (df_XGBoost["f1_std"] / np.sqrt(10))
df_XGBoost["f1_confu"] = df_XGBoost["f1_mean"]+ 1.96 * (df_XGBoost["f1_std"] / np.sqrt(10))
# get a look of the final dataset
df_XGBoost.head()

,scaler,estimator,imputer,sampler,X_shape,one_hot_encoded_features,auc_mean,execution_time_in_seconds,auc,parameters.impute__strategy,...,auc_std,pre_mean,rec_mean,f1_std,f1_mean,type2,auc_confl,auc_confu,f1_confl,f1_confu
0,MaxAbsScaler,XGBClassifier,SimpleImputer,str,"[899, 43]","[cp, restecg, slope, ca, restwm]",78.933375,42.595838,"[0.8429566948730711, 0.8494275759084122, 0.769...","[mean, median, most_frequent]",...,0.086277,0.798694,0.791823,0.085958,0.788083,93,78.879900,78.986850,0.734806,0.841361
1,MaxAbsScaler,XGBClassifier,SimpleImputer,RandomOverSampler,"[899, 43]","[cp, restecg, slope, ca, restwm]",80.348196,43.741437,"[0.8429566948730711, 0.8494275759084122, 0.779...","[mean, median, most_frequent]",...,0.094789,0.814133,0.804007,0.096549,0.798935,95,80.289445,80.406946,0.739093,0.858776
2,MaxAbsScaler,XGBClassifier,SimpleImputer,RandomUnderSampler,"[899, 43]","[cp, restecg, slope, ca, restwm]",78.623196,39.495307,"[0.8103534096565455, 0.8494275759084122, 0.833...","[mean, median, most_frequent]",...,0.088397,0.797177,0.786230,0.087638,0.782408,106,78.568407,78.677985,0.728090,0.836727
3,MinMaxScaler,XGBClassifier,SimpleImputer,str,"[899, 43]","[cp, restecg, slope, ca, restwm]",79.035416,41.588141,"[0.8429566948730711, 0.8494275759084122, 0.769...","[mean, median, most_frequent]",...,0.084202,0.800894,0.792946,0.084003,0.789013,92,78.983226,79.087605,0.736948,0.841079
4,MinMaxScaler,XGBClassifier,SimpleImputer,RandomOverSampler,"[899, 43]","[cp, restecg, slope, ca, restwm]",79.444139,43.949527,"[0.8429566948730711, 0.8494275759084122, 0.824...","[mean, median, most_frequent]",...,0.078423,0.803676,0.796255,0.077847,0.792651,94,79.395532,79.492746,0.744401,0.840901


In [36]:
# only display variables important for the model evaluation
df_XGBoost = df_XGBoost.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_XGBoost[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
8,81.217123,0.073873,85,0
5,80.944139,0.066296,99,8
20,80.656919,0.091877,88,0
17,80.630911,0.087539,102,0
6,80.420620,0.073119,77,0
7,80.369139,0.077792,86,0
1,80.348196,0.094789,95,100
19,80.181359,0.087285,93,8
11,80.167098,0.070804,103,0
14,79.826269,0.100354,94,0


In [50]:
# best model
XGBoost = df_XGBoost.loc[8]
XGBoost[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                                                                                               Normalizer
estimator                                                                                         XGBClassifier
imputer                                                                                           SimpleImputer
sampler                                                                                      RandomUnderSampler
X_shape                                                                                               [899, 43]
one_hot_encoded_features                                                       [cp, restecg, slope, ca, restwm]
auc_mean                                                                                              81.217123
execution_time_in_seconds                                                                            118.810375
auc                                                           [0.8429566948730711, 0.8554006968641116, 0

In [37]:
# loading KNeighbors 
with open("output/output KNN.json", 'r') as f:
  data_KNeighbors = json.load(f)

df_KNeighbors = pd.json_normalize(data_KNeighbors, record_path =['measurements'])
df_KNeighbors["drop_columns"] = df_KNeighbors["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_KNeighbors["auc_std"] = df_KNeighbors.apply(lambda row: np.std(row["auc"]), axis =1)
df_KNeighbors["pre_mean"] = df_KNeighbors.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_KNeighbors["rec_mean"] = df_KNeighbors.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_KNeighbors["f1_std"] = df_KNeighbors.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_KNeighbors["f1_mean"] = df_KNeighbors.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_KNeighbors["type2"] = df_KNeighbors.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_KNeighbors["auc_confl"] = df_KNeighbors["auc_mean"]- 1.96 * (df_KNeighbors["auc_std"] / np.sqrt(10))
df_KNeighbors["auc_confu"] = df_KNeighbors["auc_mean"]+ 1.96 * (df_KNeighbors["auc_std"] / np.sqrt(10))
df_KNeighbors["f1_confl"] = df_KNeighbors["f1_mean"]- 1.96 * (df_KNeighbors["f1_std"] / np.sqrt(10))
df_KNeighbors["f1_confu"] = df_KNeighbors["f1_mean"]+ 1.96 * (df_KNeighbors["f1_std"] / np.sqrt(10))
df_KNeighbors.head()
df_KNeighbors = df_KNeighbors.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_KNeighbors[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
9,84.835577,0.065546,96,8
3,84.831496,0.047235,96,8
0,84.627414,0.048671,98,8
10,84.616202,0.055622,113,8
16,84.468803,0.048838,118,20
15,84.439186,0.069498,96,20
11,84.438153,0.054576,116,8
8,84.347325,0.045372,102,0
4,84.209632,0.055243,118,8
12,83.950398,0.073852,101,8


In [59]:
KNeighbors = df_KNeighbors.loc[20]
KNeighbors[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                           str
estimator       KNeighborsClassifier
imputer                SimpleImputer
sampler           RandomUnderSampler
auc_mean                    83.19109
auc_confl                  83.166501
auc_confu                  83.215679
type2                             84
f1_mean                     0.831304
f1_confl                    0.807694
f1_confu                    0.854915
drop_columns                       0
pre_mean                    0.837904
rec_mean                     0.83196
Name: 20, dtype: object

In [39]:
# loading Random Forest Classifier
with open("output/output RandomForest.json", 'r') as f:
  data_Forest = json.load(f)

df_Forest = pd.json_normalize(data_Forest, record_path =['measurements'])
df_Forest["drop_columns"] = df_Forest["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_Forest["auc_std"] = df_Forest.apply(lambda row: np.std(row["auc"]), axis =1)
df_Forest["pre_mean"] = df_Forest.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_Forest["rec_mean"] = df_Forest.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_Forest["f1_std"] = df_Forest.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_Forest["f1_mean"] = df_Forest.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_Forest["type2"] = df_Forest.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_Forest["auc_confl"] = df_Forest["auc_mean"]- 1.96 * (df_Forest["auc_std"] / np.sqrt(10))
df_Forest["auc_confu"] = df_Forest["auc_mean"]+ 1.96 * (df_Forest["auc_std"] / np.sqrt(10))
df_Forest["f1_confl"] = df_Forest["f1_mean"]- 1.96 * (df_Forest["f1_std"] / np.sqrt(10))
df_Forest["f1_confu"] = df_Forest["f1_mean"]+ 1.96 * (df_Forest["f1_std"] / np.sqrt(10))
df_Forest.head()
df_Forest = df_Forest.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_Forest[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
13,81.909383,0.084784,97,35
3,81.909383,0.084216,97,35
9,81.882342,0.087092,96,35
16,81.882342,0.087092,96,35
0,81.707342,0.083724,99,35
6,81.682342,0.086118,98,35
12,81.089472,0.087192,99,20
18,80.605525,0.068155,134,4
19,80.539000,0.073599,113,20
20,80.471329,0.086507,134,20


In [61]:
Forest = df_Forest.loc[9]
Forest[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                             str
estimator       RandomForestClassifier
imputer                  SimpleImputer
sampler                            str
auc_mean                     81.882342
auc_confl                    81.828362
auc_confu                    81.936322
type2                               96
f1_mean                       0.814773
f1_confl                      0.762956
f1_confu                      0.866591
drop_columns                        35
pre_mean                      0.829746
rec_mean                      0.817403
Name: 9, dtype: object

In [2]:
# loading Decision Tree Classifier
with open("output/output DecisionTree.json", 'r') as f:
  data_Tree = json.load(f)

df_Tree = pd.json_normalize(data_Tree, record_path =['measurements'])
df_Tree["drop_columns"] = df_Tree["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_Tree["auc_std"] = df_Tree.apply(lambda row: np.std(row["auc"]), axis =1)
df_Tree["pre_mean"] = df_Tree.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_Tree["rec_mean"] = df_Tree.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_Tree["f1_std"] = df_Tree.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_Tree["f1_mean"] = df_Tree.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_Tree["type2"] = df_Tree.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_Tree["auc_confl"] = df_Tree["auc_mean"]- 1.96 * (df_Tree["auc_std"] / np.sqrt(10))
df_Tree["auc_confu"] = df_Tree["auc_mean"]+ 1.96 * (df_Tree["auc_std"] / np.sqrt(10))
df_Tree["f1_confl"] = df_Tree["f1_mean"]- 1.96 * (df_Tree["f1_std"] / np.sqrt(10))
df_Tree["f1_confu"] = df_Tree["f1_mean"]+ 1.96 * (df_Tree["f1_std"] / np.sqrt(10))
df_Tree.head()
df_Tree = df_Tree.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_Tree[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
7,83.005911,0.042815,87,0
8,83.005911,0.042815,87,0
1,82.880911,0.043414,87,0
4,82.880911,0.043414,87,0
10,82.880911,0.043414,87,0
13,82.880911,0.043414,87,0
16,82.880911,0.043414,87,0
19,82.880911,0.043414,87,0
2,82.790530,0.042502,77,0
5,82.790530,0.042502,77,0


In [3]:
Tree = df_Tree.loc[2]

Tree[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                    MaxAbsScaler
estimator       DecisionTreeClassifier
imputer                  SimpleImputer
sampler             RandomUnderSampler
auc_mean                      82.79053
auc_confl                    82.764187
auc_confu                    82.816873
type2                               77
f1_mean                        0.82883
f1_confl                        0.8034
f1_confu                       0.85426
drop_columns                         0
pre_mean                      0.834974
rec_mean                      0.829738
Name: 2, dtype: object

In [5]:
Tree.to_excel(r'DTree.xlsx', index=False)

In [41]:
# loading SVC
with open("output/output SVC.json", 'r') as f:
  data_SVC = json.load(f)

df_SVC = pd.json_normalize(data_SVC, record_path =['measurements'])
df_SVC["drop_columns"] = df_SVC["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_SVC["auc_std"] = df_SVC.apply(lambda row: np.std(row["auc"]), axis =1)
df_SVC["pre_mean"] = df_SVC.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_SVC["rec_mean"] = df_SVC.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_SVC["f1_std"] = df_SVC.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_SVC["f1_mean"] = df_SVC.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_SVC["type2"] = df_SVC.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_SVC["auc_confl"] = df_SVC["auc_mean"]- 1.96 * (df_SVC["auc_std"] / np.sqrt(10))
df_SVC["auc_confu"] = df_SVC["auc_mean"]+ 1.96 * (df_SVC["auc_std"] / np.sqrt(10))
df_SVC["f1_confl"] = df_SVC["f1_mean"]- 1.96 * (df_SVC["f1_std"] / np.sqrt(10))
df_SVC["f1_confu"] = df_SVC["f1_mean"]+ 1.96 * (df_SVC["f1_std"] / np.sqrt(10))
df_SVC.head()
df_SVC = df_SVC.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_SVC[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
4,85.398357,0.051551,109.0,60.0
0,85.308537,0.053953,95.0,35.0
17,85.003472,0.063505,103.0,20.0
3,84.937618,0.052429,95.0,35.0
1,84.739186,0.042993,118.0,75.0
11,84.390194,0.065996,104.0,100.0
2,83.992795,0.040578,124.0,75.0
5,83.752974,0.054579,124.0,60.0
13,83.212232,0.047015,144.0,4.0
16,82.792260,0.077671,110.0,20.0


In [63]:
SVC = df_SVC.loc[0]
SVC[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler           MaxAbsScaler
estimator                 SVC
imputer         SimpleImputer
sampler                   str
auc_mean            85.308537
auc_confl           85.275096
auc_confu           85.341977
type2                    95.0
f1_mean              0.848281
f1_confl             0.815803
f1_confu             0.880758
drop_columns             35.0
pre_mean              0.86004
rec_mean             0.848639
Name: 0, dtype: object

In [42]:
# loading naïve bayes (bernoulli)
with open("output/output BernoulliNB.json", 'r') as f:
  data_bernoulli = json.load(f)

df_bernoulli = pd.json_normalize(data_bernoulli, record_path =['measurements'])
df_bernoulli["drop_columns"] = df_bernoulli["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_bernoulli["auc_std"] = df_bernoulli.apply(lambda row: np.std(row["auc"]), axis =1)
df_bernoulli["pre_mean"] = df_bernoulli.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_bernoulli["rec_mean"] = df_bernoulli.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_bernoulli["f1_std"] = df_bernoulli.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_bernoulli["f1_mean"] = df_bernoulli.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_bernoulli["type2"] = df_bernoulli.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_bernoulli["auc_confl"] = df_bernoulli["auc_mean"]- 1.96 * (df_bernoulli["auc_std"] / np.sqrt(10))
df_bernoulli["auc_confu"] = df_bernoulli["auc_mean"]+ 1.96 * (df_bernoulli["auc_std"] / np.sqrt(10))
df_bernoulli["f1_confl"] = df_bernoulli["f1_mean"]- 1.96 * (df_bernoulli["f1_std"] / np.sqrt(10))
df_bernoulli["f1_confu"] = df_bernoulli["f1_mean"]+ 1.96 * (df_bernoulli["f1_std"] / np.sqrt(10))
df_bernoulli.head()
df_bernoulli = df_bernoulli.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_bernoulli[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
14,80.553571,0.082867,154,8
8,80.359694,0.074453,156,8
2,80.316824,0.075029,154,8
12,80.303571,0.082163,154,8
11,80.297449,0.054180,174,20
7,80.237743,0.076842,156,8
0,80.214783,0.073702,155,8
6,80.214783,0.073702,155,8
18,80.214783,0.073702,155,8
20,80.191824,0.073433,154,8


In [64]:
bernoulli = df_bernoulli.loc[2]
bernoulli[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                MaxAbsScaler
estimator              BernoulliNB
imputer              SimpleImputer
sampler         RandomUnderSampler
auc_mean                 80.316824
auc_confl                80.270321
auc_confu                80.363328
type2                          154
f1_mean                   0.786969
f1_confl                  0.742678
f1_confu                   0.83126
drop_columns                     8
pre_mean                   0.83534
rec_mean                  0.791798
Name: 2, dtype: object

In [43]:
# loading naïve bayes (categorical)
with open("output/output CategoricalNB.json", 'r') as f:
  data_categorical = json.load(f)

df_categorical = pd.json_normalize(data_categorical, record_path =['measurements'])
df_categorical["drop_columns"] = df_XGBoost["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_categorical["auc_std"] = df_categorical.apply(lambda row: np.std(row["auc"]), axis =1)
df_categorical["pre_mean"] = df_categorical.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_categorical["rec_mean"] = df_categorical.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_categorical["f1_std"] = df_categorical.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_categorical["f1_mean"] = df_categorical.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_categorical["type2"] = df_categorical.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_categorical["auc_confl"] = df_categorical["auc_mean"]- 1.96 * (df_categorical["auc_std"] / np.sqrt(10))
df_categorical["auc_confu"] = df_categorical["auc_mean"]+ 1.96 * (df_categorical["auc_std"] / np.sqrt(10))
df_categorical["f1_confl"] = df_categorical["f1_mean"]- 1.96 * (df_categorical["f1_std"] / np.sqrt(10))
df_categorical["f1_confu"] = df_categorical["f1_mean"]+ 1.96 * (df_categorical["f1_std"] / np.sqrt(10))
df_categorical.head()
df_categorical = df_categorical.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_categorical[["auc_mean", "auc_std", "type2", "drop_columns"]].head(7)

,auc_mean,auc_std,type2,drop_columns
1,81.856620,0.039589,161.0,100
0,81.454579,0.042051,165.0,8
16,NaN,NaN,NaN,75
4,NaN,NaN,NaN,60
2,NaN,NaN,NaN,20
3,NaN,NaN,NaN,20
9,NaN,NaN,NaN,20


In [65]:
categorical = df_categorical.loc[0]
categorical[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler           MaxAbsScaler
estimator       CategoricalNB
imputer         SimpleImputer
sampler                   str
auc_mean            81.454579
auc_confl           81.428516
auc_confu           81.480643
type2                   165.0
f1_mean              0.796626
f1_confl             0.769551
f1_confu             0.823701
drop_columns                8
pre_mean             0.847046
rec_mean             0.799713
Name: 0, dtype: object

In [44]:
# loading naïve bayes (complement)
with open("output/output CompleteNB.json", 'r') as f:
  data_complement = json.load(f)

df_complement = pd.json_normalize(data_complement, record_path =['measurements'])
df_complement["drop_columns"] = df_complement["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_complement["auc_std"] = df_complement.apply(lambda row: np.std(row["auc"]), axis =1)
df_complement["pre_mean"] = df_complement.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_complement["rec_mean"] = df_complement.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_complement["f1_std"] = df_complement.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_complement["f1_mean"] = df_complement.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_complement["type2"] = df_complement.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_complement["auc_confl"] = df_complement["auc_mean"]- 1.96 * (df_complement["auc_std"] / np.sqrt(10))
df_complement["auc_confu"] = df_complement["auc_mean"]+ 1.96 * (df_complement["auc_std"] / np.sqrt(10))
df_complement["f1_confl"] = df_complement["f1_mean"]- 1.96 * (df_complement["f1_std"] / np.sqrt(10))
df_complement["f1_confu"] = df_complement["f1_mean"]+ 1.96 * (df_complement["f1_std"] / np.sqrt(10))
df_complement.head()
df_complement = df_complement.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_complement[["auc_mean", "auc_std", "type2", "drop_columns"]].head(7)

,auc_mean,auc_std,type2,drop_columns
5,80.654579,0.042143,168.0,4.0
3,80.606620,0.037444,166.0,4.0
4,80.381620,0.048972,167.0,4.0
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN


In [66]:
complement = df_complement.loc[3]
complement[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler           MinMaxScaler
estimator        ComplementNB
imputer         SimpleImputer
sampler                   str
auc_mean             80.60662
auc_confl           80.583412
auc_confu           80.629828
type2                   166.0
f1_mean              0.788637
f1_confl             0.766194
f1_confu              0.81108
drop_columns              4.0
pre_mean             0.838892
rec_mean              0.79191
Name: 3, dtype: object

In [45]:
# loading naïve bayes (gaussian)
with open("output/output GaussianNB.json", 'r') as f:
  data_gaussian = json.load(f)

df_gaussian = pd.json_normalize(data_gaussian, record_path =['measurements'])
df_gaussian["drop_columns"] = df_gaussian["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_gaussian["auc_std"] = df_gaussian.apply(lambda row: np.std(row["auc"]), axis =1)
df_gaussian["pre_mean"] = df_gaussian.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_gaussian["rec_mean"] = df_gaussian.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_gaussian["f1_std"] = df_gaussian.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_gaussian["f1_mean"] = df_gaussian.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_gaussian["type2"] = df_gaussian.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_gaussian["auc_confl"] = df_gaussian["auc_mean"]- 1.96 * (df_gaussian["auc_std"] / np.sqrt(10))
df_gaussian["auc_confu"] = df_gaussian["auc_mean"]+ 1.96 * (df_gaussian["auc_std"] / np.sqrt(10))
df_gaussian["f1_confl"] = df_gaussian["f1_mean"]- 1.96 * (df_gaussian["f1_std"] / np.sqrt(10))
df_gaussian["f1_confu"] = df_gaussian["f1_mean"]+ 1.96 * (df_gaussian["f1_std"] / np.sqrt(10))
df_gaussian.head()
df_gaussian = df_gaussian.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_gaussian[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
7,80.816327,0.032285,190,35
6,80.716327,0.031743,191,35
8,80.716327,0.031743,191,20
18,80.406620,0.049993,173,8
19,80.404579,0.049931,173,8
20,80.279579,0.052841,173,4
13,80.252539,0.049066,177,8
4,80.077539,0.043472,180,8
16,80.077539,0.042306,180,8
14,80.052539,0.047049,179,8


In [67]:
gaussian = df_gaussian.loc[20]
gaussian[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                         str
estimator               GaussianNB
imputer              SimpleImputer
sampler         RandomUnderSampler
auc_mean                 80.279579
auc_confl                80.246828
auc_confu                80.312331
type2                          173
f1_mean                   0.783122
f1_confl                  0.751628
f1_confu                  0.814615
drop_columns                     4
pre_mean                  0.841607
rec_mean                  0.787428
Name: 20, dtype: object

In [46]:
# loading naïve bayes (multinomial)
with open("output/output MultinomialNB.json", 'r') as f:
  data_multinomial = json.load(f)

df_multinomial = pd.json_normalize(data_multinomial, record_path =['measurements'])
df_multinomial["drop_columns"] = df_multinomial["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_multinomial["auc_std"] = df_multinomial.apply(lambda row: np.std(row["auc"]), axis =1)
df_multinomial["pre_mean"] = df_multinomial.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_multinomial["rec_mean"] = df_multinomial.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_multinomial["f1_std"] = df_multinomial.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_multinomial["f1_mean"] = df_multinomial.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_multinomial["type2"] = df_multinomial.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_multinomial["auc_confl"] = df_multinomial["auc_mean"]- 1.96 * (df_multinomial["auc_std"] / np.sqrt(10))
df_multinomial["auc_confu"] = df_multinomial["auc_mean"]+ 1.96 * (df_multinomial["auc_std"] / np.sqrt(10))
df_multinomial["f1_confl"] = df_multinomial["f1_mean"]- 1.96 * (df_multinomial["f1_std"] / np.sqrt(10))
df_multinomial["f1_confu"] = df_multinomial["f1_mean"]+ 1.96 * (df_multinomial["f1_std"] / np.sqrt(10))
df_multinomial.head()
df_multinomial = df_multinomial.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_multinomial[["auc_mean", "auc_std", "type2", "drop_columns"]].head(5)

,auc_mean,auc_std,type2,drop_columns
3,80.90662,0.041432,168.0,4.0
5,80.90662,0.041432,168.0,4.0
4,80.45662,0.050225,170.0,8.0
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN


In [68]:
multinomial = df_multinomial.loc[3]
multinomial[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler           MinMaxScaler
estimator       MultinomialNB
imputer         SimpleImputer
sampler                   str
auc_mean             80.90662
auc_confl            80.88094
auc_confu             80.9323
type2                   168.0
f1_mean              0.790685
f1_confl             0.766195
f1_confu             0.815175
drop_columns              4.0
pre_mean             0.844182
rec_mean             0.794132
Name: 3, dtype: object

In [70]:
# loading logistic regression
with open("output/output LogisticRegression.json", 'r') as f:
  data_logistic = json.load(f)

df_logistic = pd.json_normalize(data_logistic, record_path =['measurements'])
df_logistic["drop_columns"] = df_logistic["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_logistic["auc_std"] = df_logistic.apply(lambda row: np.std(row["auc"]), axis =1)
df_logistic["pre_mean"] = df_logistic.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_logistic["rec_mean"] = df_logistic.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_logistic["f1_std"] = df_logistic.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_logistic["f1_mean"] = df_logistic.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_logistic["type2"] = df_logistic.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_logistic["auc_confl"] = df_logistic["auc_mean"]- 1.96 * (df_logistic["auc_std"] / np.sqrt(10))
df_logistic["auc_confu"] = df_logistic["auc_mean"]+ 1.96 * (df_logistic["auc_std"] / np.sqrt(10))
df_logistic["f1_confl"] = df_logistic["f1_mean"]- 1.96 * (df_logistic["f1_std"] / np.sqrt(10))
df_logistic["f1_confu"] = df_logistic["f1_mean"]+ 1.96 * (df_logistic["f1_std"] / np.sqrt(10))
df_logistic.head()
df_logistic = df_logistic.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_logistic[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
4,84.696317,0.066324,96,60
12,84.038626,0.053945,89,60
1,83.976406,0.074660,97,60
14,83.943243,0.061434,101,4
13,83.590194,0.064117,102,60
0,83.537618,0.078173,89,60
3,83.535577,0.076323,89,60
20,83.422325,0.066004,100,4
16,83.349365,0.061258,102,60
10,83.332504,0.067488,101,4


In [72]:
logistic = df_logistic.loc[9]
logistic[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler            PowerTransformer
estimator       LogisticRegression
imputer              SimpleImputer
sampler                        str
auc_mean                 83.321765
auc_confl                83.277204
auc_confu                83.366326
type2                           90
f1_mean                   0.831193
f1_confl                  0.788758
f1_confu                  0.873627
drop_columns                     4
pre_mean                  0.839554
rec_mean                   0.83196
Name: 9, dtype: object